<a href="https://colab.research.google.com/github/AlexanderModestov/deeppavlov/blob/main/%D0%9E%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8_deeppavlov_(SkillBox)_rubert_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install deeppavlov
! pip install transformers

In [1]:
import torch

# Check if CUDA is available
print("CUDA is available:", torch.cuda.is_available())

# Count of CUDA devices
print("Number of GPUs:", torch.cuda.device_count())

# Name of the current CUDA device
if torch.cuda.is_available():
    print("Current CUDA device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

CUDA is available: True
Number of GPUs: 1
Current CUDA device name: Tesla T4


In [ ]:
from deeppavlov import configs, train_model, build_model
from deeppavlov import evaluate_model
from deeppavlov.core.commands.utils import parse_config
from sklearn.model_selection import train_test_split
import json
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#%%time
#config = parse_config('/content/drive/MyDrive/ColabNotebooks/SkillBox/Configs/config_rubert_test_2.0.json')
#model = train_model(config, download=False)
#evaluate_model(config)

In [ ]:
#%%time
#config = parse_config('/content/drive/MyDrive/ColabNotebooks/SkillBox/Configs/config_rubert_test_10_2.0.json')
#model_10 = train_model(config, download=False)
#evaluate_model(config)

In [ ]:
#%%time
#config = parse_config('/content/drive/MyDrive/ColabNotebooks/SkillBox/Configs/config_rubert_test_100_2.0.json')
#model_100 = train_model(config, download=False)
#evaluate_model(config)

In [ ]:
#%%time
#config = parse_config('/content/drive/MyDrive/ColabNotebooks/SkillBox/Configs/config_rubert_test.json')
#model = build_model(config, download=False)
#evaluate_model(config)

In [ ]:
config = parse_config('/content/drive/MyDrive/ColabNotebooks/SkillBox/Configs/config_rubert_test_100_2.0.json')
model = build_model(config, download=False)
evaluate_model(config)

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/SkillBox/Data/test.csv')

def get_indexes(prob, top=3):
    sorted_indices = sorted(range(len(prob)), key=lambda k: prob[k], reverse=True)
    return sorted_indices[:top]

df = pd.DataFrame(columns=['intent', 'top_1', 'top_2', 'top_3'])
def get_top_intents(test, model, top=3):
    intents = list(model['classes_vocab'].keys())
    content, fact_intent, top_1, top_2, top_3 = [], [], [], [], []
    for index, row in test.iterrows():
        get_probs = model([row[0]])[0][0]
        selected = [intents[i] for i in get_indexes(get_probs)]
        probabilities = [get_probs[i] for i in get_indexes(get_probs)]
        print('fact: ', row[1], 'pred: ', dict(zip(selected, probabilities)))
        content.append(row[0])
        fact_intent.append(row[1])
        top_1.append(selected[0])
        top_2.append(selected[1])
        top_3.append(selected[2])
        # df.append([row[1], selected[0], selected[1], selected[2]])
        # Append the new data to the DataFrame
        print({'intent': row[1], 'top_1': selected[0], 'top_2': selected[1], 'top_3': selected[2]})
        #df = pd.concat([df, pd.DataFrame({'intent': row[1], 'top_1': selected[0], 'top_2': selected[1], 'top_3': selected[2]})], ignore_index=True)
    return pd.DataFrame({'content': content, 'fact': fact_intent, 'top_1': top_1, 'top_2': top_2, 'top_3': top_3})

In [ ]:
res = get_top_intents(test, model, top=3)

In [ ]:
res

In [ ]:
res.to_csv('/content/drive/MyDrive/ColabNotebooks/res_model_100.csv')

In [ ]:
#! python -m deeppavlov riseapi /content/drive/MyDrive/ColabNotebooks/SkillBox/Configs/config_rubert_test.json